<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/SPACE_FP_CREWAI_09JUNE25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai -q
!pip install transformers -q
!pip install bitsandbytes -q
!pip install langchain_community -q
!pip install colab-env -q

In [ ]:
!pip install crewai[tools] -q

In [ ]:
# Upgrade transformers, which should also upgrade tokenizers to a compatible version
!pip install transformers -U -q

In [ ]:
from crewai import Agent, Task, Crew, Process
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sklearn.linear_model import LogisticRegression
import re
import os # For checking file existence
import colab_env # For loading environment variables from .env file

from crewai.tools import BaseTool
import pandas as pd

# --- 1. Define Core Logic Classes (adapted from your notebook) ---
# These classes encapsulate the domain-specific logic and data handling.
# In a more complex CrewAI application, these might be wrapped as dedicated 'Tools'.

class SpaceEnvironment:
    """Manages environmental data related to space, specifically space weather."""
    def __init__(self, origin, destination, space_weather_df):
        self.origin = origin
        self.destination = destination
        self.space_weather_df = space_weather_df

    def get_possible_trajectories(self):
        """Defines predefined trajectories between celestial bodies."""
        trajectories = {
            ("Earth orbit", "Moon surface"): [
                {
                    "trajectory_id": "T1",
                    "delta_v": 3.9,
                    "duration": 3.5,
                    "fuel_required": 1200,
                    "waypoints": ["Lunar Gateway"],
                }
                # Add more trajectories here if needed, following the original notebook's intent
            ]
        }
        return trajectories.get((self.origin, self.destination), [])

    def get_space_weather_data(self, location):
        """Extracts and interprets latest space weather data for a given location."""
        if self.space_weather_df.empty:
            return {"solar_flare_risk": "Unknown", "radiation_level": "Unknown"}

        latest_data = self.space_weather_df.iloc[-1]
        # Adjusted based on the original notebook's logic for "Flares: C" and "Sunspot Number"
        # The notebook uses 'Flares: C' to determine 'solar_flare_risk' in SpaceEnvironment class
        solar_flare_risk = "High" if latest_data.get("Flares: C", 0) > 0 else "Low"
        radiation_level = "Moderate" # Simplified as per original notebook
        return {"solar_flare_risk": solar_flare_risk, "radiation_level": radiation_level}

class SpaceWeatherModelTrainer:
    """Handles training of a simple logistic regression model for high solar activity."""
    def __init__(self, space_weather_df):
        self.space_weather_df = space_weather_df
        self.model = LogisticRegression()

    def train_model(self):
        """Trains the solar weather prediction model."""
        if self.space_weather_df.empty:
            print("Cannot train model: space_weather_df is empty.")
            return

        # Feature engineering from the original notebook
        # Note: The original notebook used (Sunspot Number > 50) OR (Radio Flux 10.7cm > 150)
        # for 'high_solar_activity'
        self.space_weather_df["high_solar_activity"] = (
            (self.space_weather_df["Sunspot Number"] > 50) |
            (self.space_weather_df["Radio Flux 10.7cm"] > 150)
        ).astype(int)

        features = self.space_weather_df[["Sunspot Number", "Radio Flux 10.7cm"]]
        target = self.space_weather_df["high_solar_activity"]

        if not features.empty and not target.empty:
            self.model.fit(features, target)
            print("Space weather model trained successfully!")
        else:
            print("Insufficient data to train space weather model.")

    def predict_solar_flare_risk(self, sunspot_number, radio_flux):
        """Predicts solar flare risk based on trained model."""
        # This prediction logic is simplified; the original notebook had more steps
        # involving creating DataFrames for prediction.
        # Here we'll return 'High' if model predicts 1, 'Low' if 0.
        try:
            prediction_features = pd.DataFrame([[sunspot_number, radio_flux]],
                                               columns=["Sunspot Number", "Radio Flux 10.7cm"])
            prediction = self.model.predict(prediction_features)[0]
            return "High" if prediction == 1 else "Low"
        except Exception as e:
            print(f"Error during solar flare risk prediction: {e}")
            return "Unknown"


# --- 2. Global Setup (LLM and Data Loading) ---
# This part would typically be handled by your main script or a dedicated setup function.

DATA_PATH = '/content/gdrive/MyDrive/datasets/SW/daily_solar_data.csv'

space_weather_df = pd.DataFrame() # Initialize empty DataFrame
if os.path.exists(DATA_PATH):
    try:
        space_weather_df = pd.read_csv(DATA_PATH)
        space_weather_df['Date'] = pd.to_datetime(space_weather_df['Date'])
        print(f"Successfully loaded solar data from {DATA_PATH}.")
    except Exception as e:
        print(f"Error loading solar data: {e}. Please ensure '{DATA_PATH}' is correct and accessible.")
else:
    print(f"Solar data file not found at {DATA_PATH}. Please make sure it's available.")
    print("For demonstration, some data will be simulated if the file is not found.")
    # Simulate some data for demonstration if file not found
    space_weather_df = pd.DataFrame({
        'Date': pd.to_datetime(['1997-01-01', '1997-01-02', '2025-04-12']),
        'Radio Flux 10.7cm': [72, 72, 152],
        'Sunspot Number': [0, 0, 83],
        'Flares: C': [0, 0, 0] # Add 'Flares: C' to prevent KeyError
    })

# Initialize SpaceEnvironment and SpaceWeatherModelTrainer
environment = SpaceEnvironment("Earth orbit", "Moon surface", space_weather_df)
weather_trainer = SpaceWeatherModelTrainer(space_weather_df)
weather_trainer.train_model()


# LLM Setup (Llama 3 from Hugging Face, as per original notebook)
# This part requires significant resources (GPU) and correct library versions.
# If you don't have a strong GPU or prefer cloud LLMs, you would use a different setup here
# (e.g., from langchain_community.llms import OpenAI, or a custom class for your hosted LLM).

try:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    llama_model_id = "meta-llama/Llama-3.1-8B-Instruct" #
    llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id, use_fast=True)
    llama_tokenizer.padding_side = "right"
    llama_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    llama_model = AutoModelForCausalLM.from_pretrained(
        llama_model_id,
        device_map=device,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    llama_model.resize_token_embeddings(len(llama_tokenizer), mean_resizing=False)
    llama_model.to(device)

    llama_pipeline = pipeline(
        "text-generation",
        model=llama_model,
        tokenizer=llama_tokenizer,
        pad_token_id=llama_tokenizer.pad_token_id,
        max_new_tokens=2048, #
        do_sample=True, #
        top_k=50, #
        top_p=0.95, #
        temperature=0.7 #
    )
    print("Llama 3.1 model and pipeline initialized successfully.")
    llm_available = True
except Exception as e:
    print(f"Could not initialize Llama 3.1 model. Error: {e}")
    print("Proceeding with a placeholder LLM for plan generation.")
    llm_available = False
    # Placeholder LLM pipeline for demonstration if real LLM fails
    def placeholder_pipeline(prompt, **kwargs):
        print(f"\n--- Placeholder LLM Response for Prompt ---\n{prompt}\n---")
        return [{"generated_text": "A detailed space flight plan could not be generated due to LLM initialization issues. Please check your setup.\n\n**Mission Briefing:**\nObjective: Transport crew from Earth to Moon.\nTrajectory: T1\nSolar Flare Risk: High\n\n**Mission Plan:**\nPhase 1: Depart Earth\nPhase 2: Arrive Moon\nPhase 3: Surface Ops\nPhase 4: Return Earth\n\nThis is a placeholder plan."}]
    llama_pipeline = placeholder_pipeline

In [4]:
# --- 3. Define CrewAI Agents ---
# These agents represent the distinct roles in your flight planning team.

# Galileo: Space Environment Observer Agent
galileo = Agent(
    role='Space Environment Observer',
    goal='Observe and process real-time space weather data to assess risks for space missions.',
    backstory="""Galileo, the celestial observer, meticulously gathers and interprets
    solar activity data to inform crucial space mission decisions. His insights are
    foundational for assessing environmental hazards. """,
    verbose=False,
    allow_delegation=False,
    # Assign the LLM to the agent if using `tools` that depend on it
    # llm=llama_pipeline # Not directly assignable to agent if using custom pipeline; tools handle it.
)

# Newton: Trajectory Planning Engineer
newton = Agent(
    role='Trajectory Planning Engineer',
    goal='Design and evaluate optimal and energy-efficient space trajectories for missions, including intermediate waypoints.',
    backstory="""Newton, the master of orbital mechanics, calculates the most efficient
    and feasible paths through space, considering all gravitational forces and staging points
    like the Lunar Gateway. """,
    verbose=False,
    allow_delegation=False,
)

# Einstein: Mission Architect Agent
einstein = Agent(
    role='Mission Architect',
    goal='Integrate all environmental data and trajectory options to formulate a comprehensive, detailed, and safe space flight plan.',
    backstory="""Einstein, the grand strategist, synthesizes environmental observations
    and trajectory designs into a coherent and actionable mission blueprint. He ensures
    the plan aligns with mission objectives and safety considerations. """,
    verbose=False,
    allow_delegation=True, # Einstein might delegate internal thought processes to refine the plan
)

# Hinton: Contingency & Safety Analyst
hinton = Agent(
    role='Contingency and Safety Analyst',
    goal='Identify potential risks within the space flight plan and develop robust mitigation and emergency procedures.',
    backstory="""Hinton, the safety and resilience expert, ensures that every mission is
    equipped with thorough contingency plans and safety protocols. He meticulously
    reviews plans for system failures, solar events, and communication issues, striving for
    a highly robust and safe mission. """,
    verbose=False,
    allow_delegation=False,
)

# --- 4. Define CrewAI Tasks ---
# Each task describes a specific job for an agent.

# Helper function to generate LLM response for Einstein
def generate_llm_plan_text(chosen_trajectory, origin_weather, destination_weather):
    """Generates the detailed flight plan using the LLM."""
    prompt_template = f"""[INST] Generate a detailed space flight plan for a mission from {environment.origin} to
using NASA's Lunar Gateway as a staging point. Consider the following:
Chosen Trajectory: {chosen_trajectory}
Solar flare risk at origin: {origin_weather['solar_flare_risk']}
Solar flare risk at destination: {destination_weather['solar_flare_risk']}
# Incorporate the mission parameters: Delta-V, Duration, Fuel Required, Waypoints.
# Detail safety considerations: Solar Flare Protection, Life Support Systems, Emergency Protocols.
# Outline a comprehensive Mission Timeline with specific dates.
# Define Mission Success Criteria.
# Include a section for Mission Contingency Planning.
# Provide a plan for Mission Review and Evaluation.
# Conclude with a brief summary of the mission.
# Ensure the tone is professional and factual, suitable for a NASA mission brief.
[/INST]"""

    print("\n--- Sending prompt to LLM for plan generation ---")
    response = llama_pipeline(prompt_template, max_new_tokens=2048)
    generated_text = response[0]['generated_text']

    # Post-processing as in the original notebook
    generated_text = re.sub(r"\n+", "\n", generated_text)
    generated_text = re.sub(r"## (.*)", r"\n\n**\1**:", generated_text)
    return generated_text

# Galileo's Tasks
galileo_task_assess = Task(
    description="""Assess the current space weather conditions based on the loaded solar data.
    Determine the solar flare risk and radiation level for both Earth orbit (origin) and
    Moon surface (destination). Provide this assessment as a structured dictionary.
    Use the latest data entry in the loaded space_weather_df.
    """,
    agent=galileo,
    expected_output="A dictionary with 'origin_weather' and 'destination_weather' keys, each containing "
                    "a dictionary with 'solar_flare_risk' and 'radiation_level'."
)

# Newton's Tasks
newton_task_define_trajectories = Task(
    description="""Identify and list all predefined possible trajectories from Earth orbit
    to the Moon surface. For each trajectory, provide its ID, delta-v, duration (days),
    fuel required (kg), and any waypoints (e.g., Lunar Gateway).
    """,
    agent=newton,
    expected_output="A list of dictionaries, where each dictionary describes a possible trajectory "
                    "with keys like 'trajectory_id', 'delta_v', 'duration', 'fuel_required', 'waypoints'."
)

# Einstein's Tasks
einstein_task_formulate_plan = Task(
    description="""As the Mission Architect, you will integrate the space weather assessment
    (from Galileo) and the available trajectories (from Newton) to formulate a comprehensive
    space flight plan.
    Select the optimal trajectory (for this demo, assume the first available is the chosen one).
    Generate a detailed plan that includes:
    1.  **Mission Briefing**: Objective, Chosen Trajectory (T1), Mission Parameters (Delta-V, Duration, Fuel Required, Waypoints).
    2.  **Solar Flare Risk Assessment**: For Origin and Destination.
    3.  **Mission Plan Phases**: Detailed steps from Earth Orbit to Lunar Gateway, Lunar Gateway to Lunar Surface, Lunar Surface Operations, and Lunar Gateway to Earth Orbit. Include launch vehicles, transfer vehicles, orbits, and landing sites.
    4.  **Safety Considerations**: Solar Flare Protection, Life Support Systems, Emergency Protocols.
    5.  **Mission Timeline**: Specific dates for key events (Launch, Arrival at Gateway, Descent, Return, Re-entry).
    6.  **Mission Success Criteria**: Crew Safety, Scientific Objectives, Mission Timeline Adherence.
    7.  **Mission Contingency Planning**: For Solar Flare Events and System Failures.
    8.  **Mission Review and Evaluation**: Pre-Launch, In-Flight, Post-Mission Reviews.
    9.  **Conclusion**: Summarize the plan's safety and efficiency.
    10. **Addendum & References**.

    Your final output should be a well-structured markdown document.
    """,
    agent=einstein,
    context=[galileo_task_assess, newton_task_define_trajectories],
    expected_output="A comprehensive space flight plan in markdown format, following the specified sections."
)

# Hinton's Tasks
hinton_task_risk_review = Task(
    description="""Review the 'Generated Space Flight Plan' created by the Mission Architect (Einstein).
    Focus on explicitly identifying any gaps or weaknesses in the risk assessment,
    contingency planning, and safety protocols sections. Provide constructive feedback
    or suggest additional details that would enhance the plan's robustness.
    """,
    agent=hinton,
    context=[einstein_task_formulate_plan], # Hinton reviews Einstein's output
    expected_output="A bullet-point summary of identified risks, gaps in contingency plans, and suggestions for improvement, or a confirmation of plan robustness."
)

hinton_task_refine_contingency = Task(
    description="""Based on your review and any identified gaps, propose specific, actionable
    refinements or additions to the mission's contingency plans and safety protocols.
    Ensure these refinements address solar flares, system failures, communication, and
    lunar landing/ascent risks comprehensively.
    """,
    agent=hinton,
    context=[hinton_task_risk_review], # Builds on the risk review
    expected_output="A detailed list of refined contingency measures and safety protocols to be added to the final flight plan."
)

# --- 5. Create the Crew ---
# The Crew orchestrates the agents and tasks.

flight_planning_crew = Crew(
    agents=[galileo, newton, einstein, hinton],
    tasks=[
        galileo_task_assess,
        newton_task_define_trajectories,
        einstein_task_formulate_plan,
        hinton_task_risk_review,
        hinton_task_refine_contingency
    ],
    verbose=False, # Outputs a lot of details about the execution
    process=Process.sequential # Tasks run in the defined order
)

# --- 6. Execute the Crew ---
print("\n#################################################################################")
print("############# Initiating Flight Planning AI Agent Crew #############")
print("#################################################################################\n")
print('\n')

# Manually trigger the initial data/model interactions that the CrewAI tasks would
# rely on if they were proper tools. In a real CrewAI setup, these would be
# encapsulated within tool functions called by the agents.

# Simulate Galileo's output
galileo_output_data = environment.get_space_weather_data(environment.origin)
galileo_output_dest = environment.get_space_weather_data(environment.destination)
# Assign the output to the task for subsequent tasks to use as context
galileo_task_assess.output = {
    "origin_weather": galileo_output_data,
    "destination_weather": galileo_output_dest
}
print(f"Galileo's initial assessment (simulated): {galileo_task_assess.output}")
print('\n')

# Simulate Newton's output
newton_task_define_trajectories.output = environment.get_possible_trajectories()
print(f"Newton's initial trajectory definition (simulated): {newton_task_define_trajectories.output}")
print('\n')

# Now, trigger Einstein's task. This task will depend on the outputs of Galileo and Newton.
# The `execute` method of a task expects to be called within the Crew's `kickoff` process.
# For demonstration, we'll manually call the LLM function which Einstein's task would use.

if llm_available:
    chosen_trajectory_for_llm = newton_task_define_trajectories.output[0] if newton_task_define_trajectories.output else {}
    origin_weather_for_llm = galileo_task_assess.output.get("origin_weather", {})
    destination_weather_for_llm = galileo_task_assess.output.get("destination_weather", {})

    einstein_generated_plan = generate_llm_plan_text(
        chosen_trajectory_for_llm,
        origin_weather_for_llm,
        destination_weather_for_llm
    )
    einstein_task_formulate_plan.output = einstein_generated_plan
else:
    einstein_task_formulate_plan.output = "Placeholder plan due to LLM unavailability."

print("\n--- Einstein's Initial Plan Generated ---")
print('\n')
print(einstein_task_formulate_plan.output)
print('\n')

# Now, kick off the full crew execution.
# The tasks will automatically access the 'context' (outputs of previous tasks).
final_plan = flight_planning_crew.kickoff()

print("\n#################################################################################")
print("############################ Crew execution finished ############################")
print("#################################################################################\n")
print('\n')

print("\n--- Final Consolidated Space Flight Plan ---")
# The final_plan will be the output of the last task (hinton_task_refine_contingency)
# You might want to combine outputs of different tasks to get the full picture.
print(final_plan)
print('\n')

# Optionally, you can print the output of other key tasks
print("\n--- Summary of Hinton's Risk Review ---")
print(hinton_task_risk_review.output)
print('\n')

print("\n--- Hinton's Refined Contingency Measures ---")
print(hinton_task_refine_contingency.output)
print('\n')


#################################################################################
############# Initiating Flight Planning AI Agent Crew #############
#################################################################################



Galileo's initial assessment (simulated): {'origin_weather': {'solar_flare_risk': 'High', 'radiation_level': 'Moderate'}, 'destination_weather': {'solar_flare_risk': 'High', 'radiation_level': 'Moderate'}}


Newton's initial trajectory definition (simulated): [{'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}]



--- Sending prompt to LLM for plan generation ---

--- Einstein's Initial Plan Generated ---


[INST] Generate a detailed space flight plan for a mission from Earth orbit to
using NASA's Lunar Gateway as a staging point. Consider the following:
Chosen Trajectory: {'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}
Solar flar